In [1]:
import pandas as pd
import re
from konlpy.tag import Mecab
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [2]:
file = '/Users/um_seun/project6/한국어_단발성_대화_데이터셋.xlsx'

data = pd.read_excel(file)


In [3]:
import pickle

model = None
with open('model.pkl','rb') as pickle_file:
    model = pickle.load(pickle_file)

tokenizer = None
with open('tokenizer.pickle','rb') as pickle_file_tok:
    tokenizer = pickle.load(pickle_file_tok)





Metal device set to: Apple M1 Pro


2022-07-05 16:23:44.211383: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-07-05 16:23:44.211512: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
#data = df['Sentence']   # Sentence == comment_ori


In [10]:
def emoticon_del(data):
    #이모티콘 제거
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    #분석에 어긋나는 불용어구 제외 (특수문자, 의성어)
    han = re.compile(r'[ㄱ-ㅎㅏ-ㅣ!?~,"a-zA-z0-9:.\n\r#\ufeff\u200d]')

    comment_list = []
    for i in range(len(data)):
        comment_list.append(data['Sentence'].iloc[i])

    comment_result = []
    for i in comment_list:
        tokens = re.sub(emoji_pattern,"",str(i))
        tokens = re.sub(han,"",tokens)
        comment_result.append(tokens)
    comment_result = pd.DataFrame(comment_result, columns=["Sentence"])
    data['Sentence'] = comment_result 

    return data


def token(data):
    # 불용어(포함 안 시킬 단어) 처리 
    stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게', '만', '게임', '겜', '되', '음', '면','!', '.', ',', '~', '?', 'ㅡ', 'ㅠ', 'ㅜ','/', ' '] 
            #'도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게', '만', '게임', '겜', '되', '음', '면',
    mecab = Mecab()
    
    data['tokenized'] = data['Sentence'].apply(mecab.morphs)
    data['tokenized'] = data['tokenized'].apply(lambda x: [item for item in x if item not in stopwords])
    
def lav_1(x):
    if x == 0:
        return '불안'
    if x == 1:
        return '당황'
    if x == 2:
        return '분노'
    if x == 3:
        return '슬픔'
    if x == 4:
        return '기쁨'
    if x == 5:
        return '기쁨'
    if x == 6:
        return '상처'
    

def tokenizer_sequences_pad_sequences(data):
    max_len = 50
    
    test_set = tokenizer.texts_to_sequences(data['tokenized'])   # Sentence == comment_ori
    pad_test_set = pad_sequences(test_set, maxlen=max_len, padding='post') #패딩 처리
    
    predict_test_set = model.predict(pad_test_set)          #모델 predict
    
    classes = np.argmax(predict_test_set, axis = 1)
    classs = pd.Series(classes)
    
    data['predict'] = classs                       
    data['predict'] = data['predict'].apply(lav_1)
    return data
    


In [7]:
#최종
def predict(data):
    data = emoticon_del(data)
    token(data)
    data = tokenizer_sequences_pad_sequences(data)
    return data

In [8]:
data = predict(data)

2022-07-05 16:24:24.752557: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-07-05 16:24:25.763633: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-05 16:24:26.027403: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-05 16:24:26.038077: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-05 16:24:26.185970: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-05 16:24:26.194316: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-05 16:24:26.315639: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113

1207/1207 [==============================] - 74s 60ms/step


In [9]:
data[['Sentence','Emotion','predict']]

,Sentence,Emotion,predict
0,언니 동생으로 부르는게 맞는 일인가요,공포,불안
1,그냥 내 느낌일뿐겠지,공포,슬픔
2,아직너무초기라서 그런거죠,공포,불안
3,유치원버스 사고 낫다던데,공포,불안
4,근데 원래이런거맞나요,공포,불안
...,...,...,...
38589,솔직히 예보 제대로 못하는 데 세금이라도 아끼게 그냥 폐지해라,혐오,분노
38590,재미가 없으니 망하지,혐오,기쁨
38591,공장 도시락 비우생적임 아르바이트했는데 화장실가성 손도 않씯고 재료 담고 바닥 떨어...,혐오,불안
38592,코딱지 만한 나라에서 지들끼리 피터지게 싸우는 센징 클래스,혐오,상처
